In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 42.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f5d19ba026999987e89d7e23b23cc90a32d84dbae4baa7ca75426f814a8aa8b1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import time
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/New Dataset/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: double (nullable = true)
 |-- mfgr: double (nullable = true)
 |-- mfri: do

In [ ]:
import pyspark.sql.functions as func
df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))
df.groupBy('redshift').count().show()

+--------+------+
|redshift| count|
+--------+------+
|       1| 48930|
|       3| 71008|
|       2| 31638|
|       0|409778|
|       6|  5347|
|       5|  6185|
|       4|  9264|
|       7|    81|
+--------+------+



In [ ]:
df = df.drop('Unnamed: 0', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1.306018|  3.220677|  0.761024|  1.247238|19.67306|  18.944|18.55688|22

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|objid|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  1.306018|  3.220677|  0.761024|  1.247238|19.67306|  

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 
                                         'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 
                                         'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [ ]:
output = assembler.transform(df)
final_df = output.select('features', 'redshift')
final_df.show(5)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[23.2391,21.44343...|       0|
|[24.05348,21.2472...|       0|
|[26.32478,20.7405...|       0|
|[24.76801,19.3207...|       0|
|[21.75992,19.0856...|       0|
+--------------------+--------+
only showing top 5 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.ml import Pipeline
nb = NaiveBayes(labelCol="redshift", featuresCol="features")

In [ ]:
start_time = time.time()
model = nb.fit(train)
print("%s seconds" % (time.time() - start_time))

17.579627752304077 seconds


In [ ]:
pred_test = model.transform(test)

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Naive Bayes Accu:', multi_evaluator.evaluate(pred_test))

Naive Bayes Accu: 0.739647132020115


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(pred_test))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(pred_test))

test data (weightedPrecision):  0.758138603512776
test data (weightedRecall):  0.7396471320201149


#Cascading

In [ ]:
final_pred = model.transform(final_df)
final_pred.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[23.2391,21.44343...|       0|[-1353.4869441199...|[0.51945174049222...|       0.0|
|[24.05348,21.2472...|       0|[-1346.2596578174...|[0.63524539210051...|       0.0|
|[26.32478,20.7405...|       0|[-1375.1791032793...|[0.99849104288834...|       0.0|
|[24.76801,19.3207...|       0|[-1317.5786455591...|[0.99985357602885...|       0.0|
|[21.75992,19.0856...|       0|[-1307.0058552724...|[0.99998808941947...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
final_pred = final_pred.withColumn("objid", monotonically_increasing_id())
final_pred.show(5)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[23.2391,21.44343...|       0|[-1353.4869441199...|[0.51945174049222...|       0.0|    0|
|[24.05348,21.2472...|       0|[-1346.2596578174...|[0.63524539210051...|       0.0|    1|
|[26.32478,20.7405...|       0|[-1375.1791032793...|[0.99849104288834...|       0.0|    2|
|[24.76801,19.3207...|       0|[-1317.5786455591...|[0.99985357602885...|       0.0|    3|
|[21.75992,19.0856...|       0|[-1307.0058552724...|[0.99998808941947...|       0.0|    4|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 5 rows



In [ ]:
print(df.count(), len(df.columns))
new_df = df.join(final_pred, df.objid == final_pred.objid).select(df["*"],final_pred["prediction"])
print(df.count(), len(df.columns))
new_df.show(5)

582231 27
582231 27
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|petroR90_z|       r|       i|       z|     mmug|     mmgr|     mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|redshift|objid|prediction|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----+----------+
|   23.2391|  21.44343|  19.67306|    18.944|  18.55688|  23.69614|  22.34922|  20.59603|  19.86871|  19.45047|  

In [ ]:
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 
                                         'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 
                                         'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [ ]:
output = assembler.transform(new_df)
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[23.2391,21.44343...|       0|
|[24.05348,21.2472...|       0|
|[25.42423,21.6029...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
nb = NaiveBayes(labelCol="redshift", featuresCol="features2")

In [ ]:
start_time = time.time()
model = nb.fit(train)
print("%s seconds" % (time.time() - start_time))

35.92470622062683 seconds


In [ ]:
pred_test2 = model.transform(test)

In [ ]:
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Naive Bayes Accu:', multi_evaluator.evaluate(pred_test2))

Naive Bayes Accu: 0.7394475277064912


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(pred_test2))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(pred_test2))

test data (weightedPrecision):  0.759181322623158
test data (weightedRecall):  0.7394475277064912
